In [1]:
import re
import string
from operator import * 
from itertools import *

from modelYelp import *
session = getSession()

from playgroundYelp import *

import numpy as np

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
STOPWORDS = set(stopwords.words('english')) - set(['not'])

mapmap = lambda f1, f2, L: map(f1, map(f2, L))
formap = lambda f, L: [map(f, _L) for _L in L]
forfilter = lambda f, L: [filter(f, _L) for _L in L]

winsize = 10
charwinsize = winsize * 13
alpha = 1.0 - (0.1**((winsize - 1) ** -1))

In [2]:
def getMostFrequentWords(city, limit=1000):
    words = session.execute('''
        SELECT token, COUNT(token)
        FROM index WHERE city = :city AND "isName" = false
        GROUP BY token ORDER BY Count DESC LIMIT :limit;
    ''', {'city': 'Pittsburgh', 'limit': 1000})

    return map(itemgetter(0), words)

words = getMostFrequentWords('Pittsburgh')

In [3]:
def bow2vec(tokens):
    total = 0.0
    vectors = []
    for offset, token in enumerate(tokens):
        if not token in model:
            continue
        offset += 1
        weight = (1-alpha) ** (offset - 1)
        total += weight
        vectors.append(model[token] * weight)
    vectors = np.asarray(vectors)
    if total == 0.0:
        return []
    return vectors.sum(0)/total


In [4]:
def snippets2vector(snippets):
    snippets = map(word_tokenize, snippets)
    for index in range(0, len(snippets), 2):
        snippets[index] = list(reversed(snippets[index]))
    snippets = [snippet[:-1] for snippet in snippets]
    snippets = formap(string.lower, snippets)
    snippets = forfilter(lambda token: len(token) >= 3 and (not token in STOPWORDS), snippets)
    snippets = filter(len, snippets)
    vectors = map(bow2vec, snippets)
    vectors = filter(len, vectors)
    vectors = np.asarray(vectors)
    return vectors.sum(0)/float(len(vectors))


In [5]:
def getWordVectorsForBusiness(city, word):
    index = session.execute('''
        SELECT business_id, review_id, start, "end" FROM index
        WHERE city = :city AND "isName" = false AND token = :word ORDER BY business_id
    ''', {'city': city, 'word': word})

    payload = []
    for business_id, bgroup in groupby(index, key=itemgetter(0)):
        mentions = []
        for review_id, rgroup in groupby(bgroup, itemgetter(1)):
            substrings = []
            for start, end in map(itemgetter(2, 3), rgroup):
                substrings.append('substring(text, %d, %d)' % (max(0, start - charwinsize), charwinsize))
                substrings.append('substring(text, %d, %d)' % (end+1, charwinsize+1))
            substring = ', '.join(substrings)
            mention = session.execute('''
                SELECT %s
                FROM review WHERE city = :city and review_id = :review_id
            ''' % substring, {'city': city, 'review_id': review_id})
            mention = reduce(add, mention)
            mentions += mention
        vector = snippets2vector(mentions)
        payload.append({
            'token': word,
            'business_id': business_id,
            'vector': vector.tolist()
        })
    return payload




In [6]:
city = 'Pittsburgh'
limit = 1000
for word in getMostFrequentWords(city, limit):
    print word
    payload = getWordVectorsForBusiness(city, word)
    session.bulk_insert_mappings(BusinessVector, payload)
    session.commit()

food
good
place
order
like
great
time
get
one
would
servic
tri
realli
restaur
back
also
menu
love
nice
got
even
delici
come
pittsburgh
fri
bar
wait
chicken
tabl
drink
pizza
best
well
make
eat
could
littl
sauc
tast
want
chees
price
look
came
beer
sandwich
meal
dish
pretti
flavor
salad
definit
much
alway
first
night
went
burger
friend
thing
think
dinner
two
never
side
peopl
made
lunch
say
friendli
experi
ask
enjoy
fresh
better
serv
seat
take
recommend
staff
know
bit
amaz
way
top
give
small
though
lot
meat
day
visit
ever
seem
everyth
favorit
minut
right
special
server
star
go
still
around
cook
atmospher
sure
bread
review
someth
sinc
said
select
hour
bad
hot
feel
soup
see
mani
need
excel
area
expect
disappoint
busi
work
tasti
taco
use
appet
option
portion
next
everi
perfect
locat
sweet
new
anoth
waitress
differ
find
stop
enough
last
open
reason
dine
walk
took
worth
long
big
roll
awesom
spici
potato
happi
check
start
dessert
howev
noth
steak
plate
qualiti
pork
end
quit
probabl
overal
larg
y

ERROR:root:An unexpected error occurred while tokenizing input
The following traceback may be corrupted or invalid
The error message is: ('EOF in multi-line string', (1, 12))



KeyboardInterrupt: 